# 5장 트랜잭션
## op_hash160 연산함수를 작성하시오.

In [1]:
import hashlib
def hash160(data):
    sha256 = hashlib.sha256(data).digest()
    ripemd160 = hashlib.new('ripemd160')
    ripemd160.update(sha256)
    return ripemd160.digest()
    
def op_hash160(stack):
    if len(stack)<1:
        return False
    element = stack.pop()
    h160=hash160(element)
    stack.append(h160)
    return True

In [2]:
stack = ["hello world".encode()]
op_hash160(stack)
print(stack)

[b'\xd7\xd5\xeex$\xff\x93\xf9L0U\xaf\x93\x82\xc8lh\xb5\xca\x92']


# 파싱함수 작성하기

In [3]:
class Script:
    def __init__(self, cmds=None):
        if cmds is None:
            self.cmds =[]
        else:
            self.cmds=cmds

    @classmethod
    def parse(cls,s):
        length = read_carint(s)
        cmds=[]
        count=0
        while count<length:
            current = s.read(1)
            count+=1
            current_byte = current[0]
            if current_byte>=1 and current_byte<=75:
                n=current_byte
                cmds.append(s.read(n))
                count+=n
            elif current_byte == 76:
                data_length=little_endian_to_int(s.read(1))
                cmds.append(s.read(data_length))
                count+=data_length+1
            elif current_byte==77:
                data_length = little_endian_to_int(s.read(2))
                cmds.append(s.read(data_length))
                count +=data_length+2
            else:
                op_code = current_byte
                cmds.append(op_code)
        if count!= length:
            raise SyntaxError('parsing script failed')
        return cls(cmds)
                

In [4]:
class Script:
    def raw_serialize(self):
        result = b''
        for cmd in self.cmds:
            if type(cmd)==int:
                result += int_to_little_endian(cmd,1)
            else:
                length = len(cmd)
            if legth<75:
                result +=int_to_little_endian(length,1)
            elif length>75 and length<0x100:
                result +=int_to_little_endian(76,1)
                result += int_to_little_endian(length,1)
            elif length>=0x100 and length<=520:
                result +=int_to_little_endian(77,1)
                result +=int_to_little_endian(length,2)
            else:
                raise ValueError('too long cmd')
            result +=cmd
        return result
    def serialize(self):
        result = self.raw_serialize()
        total=len(result)
        return encode_varint(total)+result

In [5]:
from script import Script
z = 0x7c076ff316692a3d7eb3c3bb0f8b1488cf72e1afcd929e29307032997a838a3d
sec = bytes.fromhex('04887387e452b8eacc4acfde10d9aaf7f6d9a0f975aabb10d006e\
4da568744d06c61de6d95231cd89026e286df3b6ae4a894a3378e393e93a0f45b666329a0ae34')
sig = bytes.fromhex('3045022000eff69ef2b1bd93a66ed5219add4fb51e11a840f4048\
76325a1e8ffe0529a2c022100c7207fee197d27c618aea621406f6bf5ef6fca38681d82b2f06fd\
dbdce6feab601')

In [6]:
script_pubkey = Script([sec, 0xac])

In [7]:
script_sig = Script([sig])

In [8]:
combined_script = script_sig + script_pubkey

In [9]:

print(combined_script.evaluate(z))

True


## 연습문제 6.2. op.py 파일에서 연산함수 op_checksig를 작성하시오.

In [10]:
def op_checksig(stack, z):
    
    if len(stack) < 2:
        return False

    # 2. 스택에서 공개키와 서명을 추출
    sec_pubkey = stack.pop()
    der_signature = stack.pop()

    # 3. 서명의 마지막 바이트(hash_type) 제거
    der_signature = der_signature[:-1]

    try:
        # 4. SEC 형식의 공개키를 객체로 변환
        pubkey = S256Point.parse(sec_pubkey)
        # 5. DER 서명을 Signature 객체로 변환
        signature = Signature.parse(der_signature)

        # 6. 서명 검증
        if pubkey.verify(z, signature):
            stack.append(1)  # 검증 성공
        else:
            stack.append(0)  # 검증 실패
    except (ValueError, SyntaxError):
        stack.append(0)  # 변환 실패 시 검증 실패로 간주

    return True

## 연습문제 6.3. 다음 잠금 스크립트를 해제하는 해제 스크립트를 작성하시오. op_mul은 스택 위 2개의 원소를 가져와서 곱한 결과를 스택 위로 올림

In [12]:
from script import Script
script_pubkey = Script([0x76,0x76,0x95,0x93,0x56,0x87])
script_sig=Script([0x52])
combined_script=script_sig+script_pubkey
print(combined_script.evaluate(0))

True


## 연습문제 6.4. 다음 스크립트가 어떤일을 하는 지 확인하시오.(script.parse와 스크립트에서 오피코드를 확인하시오.)
- 6e879169a77ca787

In [14]:
from script import Script
script_pubkey = Script([0x6e, 0x87, 0x91, 0x69, 0xa7, 0x7c, 0xa7, 0x87])

In [17]:
c1 = '255044462d312e330a25e2e3cfd30a0a0a312030206f626a0a3c3c2f576964746820\
32203020522f4865696768742033203020522f547970652034203020522f537562747970652035\
203020522f46696c7465722036203020522f436f6c6f7253706163652037203020522f4c656e67\
74682038203020522f42697473506572436f6d706f6e656e7420383e3e0a73747265616d0affd8\
fffe00245348412d3120697320646561642121212121852fec092339759c39b1a1c63c4c97e1ff\
fe017f46dc93a6b67e013b029aaa1db2560b45ca67d688c7f84b8c4c791fe02b3df614f86db169\
0901c56b45c1530afedfb76038e972722fe7ad728f0e4904e046c230570fe9d41398abe12ef5bc\
942be33542a4802d98b5d70f2a332ec37fac3514e74ddc0f2cc1a874cd0c78305a215664613097\
89606bd0bf3f98cda8044629a1'

In [18]:
c2 = '255044462d312e330a25e2e3cfd30a0a0a312030206f626a0a3c3c2f576964746820\
32203020522f4865696768742033203020522f547970652034203020522f537562747970652035\
203020522f46696c7465722036203020522f436f6c6f7253706163652037203020522f4c656e67\
74682038203020522f42697473506572436f6d706f6e656e7420383e3e0a73747265616d0affd8\
fffe00245348412d3120697320646561642121212121852fec092339759c39b1a1c63c4c97e1ff\
fe017346dc9166b67e118f029ab621b2560ff9ca67cca8c7f85ba84c79030c2b3de218f86db3a9\
0901d5df45c14f26fedfb3dc38e96ac22fe7bd728f0e45bce046d23c570feb141398bb552ef5a0\
a82be331fea48037b8b5d71f0e332edf93ac3500eb4ddc0decc1a864790c782c76215660dd3097\
91d06bd0af3f98cda4bc4629b1'

In [19]:
collision1 = bytes.fromhex(c1)
collision2 = bytes.fromhex(c2)

In [20]:
script_sig = Script([collision1, collision2])
combined_script = script_sig + script_pubkey
print(combined_script.evaluate(0))

True
